In [1]:
# Minhyuk Sung (mhsung@kaist.ac.kr)

from pointnet import PointNetCls

import easydict
import h5py
import numpy as np
import os
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Check whether GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
# Set hyperparameters.
args = easydict.EasyDict({
    'train': False,
    'batch_size': 32,       # input batch size
    'n_epochs': 50,         # number of epochs
    'n_workers': 4,         # number of data loading workers
    'learning_rate': 0.001, # learning rate
    'beta1': 0.9,           # beta 1
    'beta2': 0.999,         # beta 2
    'step_size': 20,        # step size
    'gamma': 0.5,           # gamma
    'in_data_file': 'data/ModelNet/modelnet_classification.h5', # data directory
    'model': 'outputs/model_50.pth',                            # model path
    'out_dir': 'outputs'    # output directory
})

In [3]:
# Define 'Dataset' class.
class Dataset(torch.utils.data.Dataset):
    def __init__(self, point_clouds, class_ids):
        # point_clouds: (N, 3)
        self.point_clouds = torch.from_numpy(point_clouds).float()
        # class_ids: (N)
        self.class_ids = torch.from_numpy(class_ids).long()

    def __len__(self):
        return np.shape(self.point_clouds)[0]

    def __getitem__(self, idx):
        return self.point_clouds[idx], self.class_ids[idx]

In [4]:
# Load the data and create dataloaders.
def create_datasets_and_dataloaders():
    assert(os.path.exists(args.in_data_file))
    f = h5py.File(args.in_data_file, 'r')

    train_data = Dataset(f['train_point_clouds'][:], f['train_class_ids'][:])
    test_data = Dataset(f['test_point_clouds'][:], f['test_class_ids'][:])

    n_classes = np.amax(f['train_class_ids']) + 1
    print('# classes: {:d}'.format(n_classes))

    train_dataloader = torch.utils.data.DataLoader(
        train_data,
        batch_size=args.batch_size,
        shuffle=args.train,
        num_workers=int(args.n_workers))

    test_dataloader = torch.utils.data.DataLoader(
        test_data,
        batch_size=args.batch_size,
        shuffle=args.train,
        num_workers=int(args.n_workers))

    return train_data, train_dataloader, test_data, test_dataloader, n_classes

In [5]:
# Define the cross entropy loss function.
def compute_loss(points, gt_classes, pred_class_logits):
    # points: (batch_size, n_points, dim_input)
    # gt_classes: (batch_size)
    # pred_class_logits: (batch_size, n_classes)
    loss = F.cross_entropy(input=pred_class_logits, target=gt_classes)
    return loss

In [6]:
# Define the accuracy function.
def compute_accuracy(points, gt_classes, pred_class_logits):
    # points: (batch_size, n_points, dim_input)
    # gt_classes: (batch_size)
    # pred_class_logits: (batch_size, n_classes)
    pred_classes = pred_class_logits.max(1)[1]
    acc = float(pred_classes.eq(gt_classes).sum()) / gt_classes.size()[0] * 100
    return acc

In [7]:
# Define one-step training function.
def run_train(data, net, optimizer, writer=None):
    # Parse data.
    points, gt_classes = data
    points = points.cuda()
    gt_classes = gt_classes.cuda().squeeze()
    # points: (batch_size, n_points, dim_input)
    # gt_classes: (batch_size)

    # Reset gradients.
    # https://pytorch.org/tutorials/recipes/recipes/zeroing_out_gradients.html#zero-the-gradients-while-training-the-network
    optimizer.zero_grad()

    # Predict.
    pred_class_logits = net.train()(points)

    # Compute the loss.
    loss = compute_loss(points, gt_classes, pred_class_logits)

    with torch.no_grad():
        # Compute the accuracy.
        acc = compute_accuracy(points, gt_classes, pred_class_logits)

    # Backprop.
    loss.backward()
    optimizer.step()

    return loss, acc

In [8]:
# Define one-step evaluation function.
def run_eval(data, net, optimizer, writer=None):
    # Parse data.
    points, gt_classes = data
    points = points.cuda()
    gt_classes = gt_classes.cuda().squeeze()
    # points: (batch_size, n_points, dim_input)
    # gt_classes: (batch_size)

    with torch.no_grad():
        # Predict.
        pred_class_logits = net.eval()(points)

        # Compute the loss.
        loss = compute_loss(points, gt_classes, pred_class_logits)

        # Compute the accuracy.
        acc = compute_accuracy(points, gt_classes, pred_class_logits)

    return loss, acc

In [9]:
# Define one-epoch training/evaluation function.
def run_epoch(dataset, dataloader, train, epoch=None, writer=None):
    total_loss = 0.0
    total_acc = 0.0
    n_data = len(dataset)

    # Create a progress bar.
    pbar = tqdm(total=n_data, leave=False)

    mode = 'Train' if train else 'Test'
    epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
            str(epoch).zfill(len(str(args.n_epochs))), args.n_epochs)

    for i, data in enumerate(dataloader):
        # Run one step.
        loss, acc = run_train(data, net, optimizer, writer) if train else \
                run_eval(data, net, optimizer, writer)

        if train and writer is not None:
            # Write results if training.
            assert(epoch is not None)
            step = epoch * len(dataloader) + i
            writer.add_scalar('Loss/Train', loss, step)
            writer.add_scalar('Accuracy/Train', acc, step)

        batch_size = list(data[0].size())[0]
        total_loss += (loss * batch_size)
        total_acc += (acc * batch_size)

        pbar.set_description('{} {} Loss: {:f}, Acc : {:.2f}%'.format(
            epoch_str, mode, loss, acc))
        pbar.update(batch_size)

    pbar.close()
    mean_loss = total_loss / float(n_data)
    mean_acc = total_acc / float(n_data)
    return mean_loss, mean_acc

In [10]:
# Define one-epoch function for both training and evaluation.
def run_epoch_train_and_test(
    train_dataset, train_dataloader, test_dataset, test_dataloader, epoch=None,
        writer=None):
    train_loss, train_acc = run_epoch(
        train_dataset, train_dataloader, train=args.train, epoch=epoch,
        writer=writer)
    test_loss, test_acc = run_epoch(
        test_dataset, test_dataloader, train=False, epoch=epoch, writer=None)

    if writer is not None:
        # Write test results.
        assert(epoch is not None)
        step = (epoch + 1) * len(train_dataloader)
        writer.add_scalar('Loss/Test', test_loss, step)
        writer.add_scalar('Accuracy/Test', test_acc, step)

    epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
            str(epoch).zfill(len(str(args.n_epochs))), args.n_epochs)

    log = epoch_str + ' '
    log += 'Train Loss: {:f}, '.format(train_loss)
    log += 'Train Acc: {:.2f}%, '.format(train_acc)
    log += 'Test Loss: {:f}, '.format(test_loss)
    log += 'Test Acc: {:.2f}%.'.format(test_acc)
    print(log)

In [11]:
# Main function.
if __name__ == "__main__":
    print(args)

    # Load datasets.
    train_dataset, train_dataloader, test_dataset, test_dataloader, \
        n_classes = create_datasets_and_dataloaders()

    # Create the network.
    n_dims = 3
    net = PointNetCls(n_dims, n_classes)
    if torch.cuda.is_available():
        net.cuda()

    # Load a model if given.
    if args.model != '':
        net.load_state_dict(torch.load(args.model))

    # Set an optimizer and a scheduler.
    optimizer = torch.optim.Adam(
        net.parameters(), lr=args.learning_rate,
        betas=(args.beta1, args.beta2))
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=args.step_size, gamma=args.gamma)

    # Create the output directory.
    if not os.path.exists(args.out_dir):
        os.makedirs(args.out_dir)

    # Train.
    if args.train:
        writer = SummaryWriter(args.out_dir)

        for epoch in range(args.n_epochs):
            run_epoch_train_and_test(
                train_dataset, train_dataloader, test_dataset, test_dataloader,
                epoch, writer)

            if (epoch + 1) % 10 == 0:
                # Save the model.
                model_file = os.path.join(
                    args.out_dir, 'model_{:d}.pth'.format(epoch + 1))
                torch.save(net.state_dict(), model_file)
                print("Saved '{}'.".format(model_file))

            scheduler.step()

        writer.close()
    else:
        run_epoch_train_and_test(
            train_dataset, train_dataloader, test_dataset, test_dataloader)

{'train': False, 'batch_size': 32, 'n_epochs': 50, 'n_workers': 4, 'learning_rate': 0.001, 'beta1': 0.9, 'beta2': 0.999, 'step_size': 20, 'gamma': 0.5, 'in_data_file': 'data/ModelNet/modelnet_classification.h5', 'model': 'outputs/model_50.pth', 'out_dir': 'outputs'}
# classes: 40


 Train Loss: 0.079747, Train Acc: 97.13%, Test Loss: 0.418972, Test Acc: 88.45%.
